In [1]:
library(dplyr)
library(readr)
library(foreign)
library(gridExtra)
library(ggthemes)
library(ggplot2)
pacman::p_load(tidyverse, haven, sjlabelled, janitor, srvyr)


Attaching package: 'dplyr'


The following objects are masked from 'package:stats':

    filter, lag


The following objects are masked from 'package:base':

    intersect, setdiff, setequal, union



Attaching package: 'gridExtra'


The following object is masked from 'package:dplyr':

    combine




In [2]:
SDEM <- read_dta("/R/ENOE_SDEMT124.dta") # Archivo .dta para leer etiquetas correspondiente al primer trimestre de 2024

In [3]:
base_des <- SDEM %>% # Descripción de las variables de la encuesta: https://www.inegi.org.mx/contenidos/programas/enoe/15ymas/doc/enoe_n_fd_c_bas_amp.pdf
  select(c(sex, eda, niv_ins, cd_a, # características de la población
           rama_est1, clase2, clase1, rama_est2, rama, seg_soc, scian, clase3, nac_anio, # sector de ocupación
           c_ocu11c, emp_ppal, emple7c, tue2, tue1, eda7c, t_tra, ambito2, cs_p13_1, # condiciones laborales
           ing7c, ingocup, ing_x_hrs, hrsocup, dur9c, tue_ppal, eda19c, remune2c, tcco, # nivel de ingresos
           fac_tri, upm, ent, mun, c_res, pos_ocu)) # factor de expansión

In [4]:
base_des1 <- base_des %>% # nueva base con población ocupada mayor a 14 años que reportan ingresos
  filter(eda > 14 & clase2 == 1 & pos_ocu != 4 & ingocup <60000 & ingocup > 0) %>% # población ocupada con ingreso mensual menor a los 60,000 pesos
  mutate_at(vars(emp_ppal), as_label) # agrupar por condición de formalidad

In [5]:
Formalidad <- factor(base_des1$emp_ppal)

cols <- c("#00FFFF", "#99CC00")

In [21]:
# Gráfico para conocer la distribución del ingreso mensual según condición de formalidad
p <- ggplot(base_des1, aes(x=ingocup, fill = Formalidad)) + geom_density(color="black",alpha=.5) + scale_fill_manual(values = cols) + 
  theme_solarized(light=FALSE) +
  ggtitle("Distribución del ingreso de la población ocupada\npor condición de formalidad") + 
  xlab("Ingreso mensual") + ylab("Densidad") + 
  theme(legend.position = c(.8, .8), legend.justification = c("right", "top"),
        legend.box.just = "right", legend.margin = margin(6,6,6,6)) +
  labs(caption = "Fuente: Encuesta Nacional de Ocupación y Empleo al 1T-24") +
  theme(plot.caption = element_text(size = 7.5, hjust = 0))

ggsave("figure1.png", dpi=300, dev='png', width=7.8, height=5.5)

In [23]:
base_des %>% # Ingreso mediano de la población ocupada en la formalidad e informalidad
 filter(eda>14 & clase2==1 & pos_ocu != 4 & ingocup>0) %>% 
 mutate_at(vars(emp_ppal), as_label) %>%
 group_by(as_label(emp_ppal)) %>%
 summarise(mediana=median(ingocup))

as_label(emp_ppal),mediana
<fct>,<dbl>
Empleo informal,6450
Empleo formal,10105
